In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 設定目前程式資料夾路徑(務必自行更改)

In [1]:
folder="/content/drive/MyDrive/解密AI黑盒子分享/"

## import 用到的套件

In [2]:
import time
import numpy as np
from IPython import display
import matplotlib.pyplot as plt
import matplotlib.font_manager as plt_font
twfont1 = plt_font.FontProperties(fname=folder+"字型/kaiu.ttf")

## 載入pytorch相關套件

In [3]:
import torch
from torch.utils.data import Dataset,DataLoader 

## 是否有GPU可以使用

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 建立訓練和測試資料集的DataSet類別

In [5]:
class MyDataset(Dataset):
  def __init__(self,Train=True): 
    if Train:
      data=np.loadtxt(folder+"資料集/MNIST/mnist_train.csv",delimiter=",")
    else:
      data=np.loadtxt(folder+"資料集/MNIST/mnist_test.csv",delimiter=",")
    self.data=torch.tensor(data[:,1:]/255).float()
    self.label=torch.tensor(data[:,0]).long()
  def __getitem__(self, index): 
    return self.data[index], self.label[index]
  def __len__(self): 
    return len(self.data)

## 實體化訓練和測試DataSet和DataLoader

In [6]:
TrainDS=MyDataset(True)
TrainDL=torch.utils.data.DataLoader(dataset=TrainDS,batch_size=100,shuffle=True)
TestDS=MyDataset(False)
TestDL=torch.utils.data.DataLoader(dataset=TestDS,batch_size=len(TestDS),shuffle=False)

## 建立RNN神經網路類別

In [7]:
class GRU(torch.nn.Module):
  def __init__(self, InputNode, HiddenNode, LayerNum, OutputNode):
    super(GRU, self).__init__()
    self.Cell = torch.nn.GRU(InputNode, HiddenNode, LayerNum, batch_first=True)
    self.fc = torch.nn.Linear(HiddenNode, OutputNode)
  def forward(self, x):
    Cellout,hx=self.Cell(x, None)
    output= self.fc(Cellout[:,-1,:]) 
    return output

## 實體化神經網路，指定損失函數和優化器

In [8]:
InputNode=28
HiddenNode=80
LayerNum=1
OutputNode=10
Net=GRU(InputNode, HiddenNode, LayerNum, OutputNode).to(device)
LossFun= torch.nn.CrossEntropyLoss()
optim = torch.optim.SGD(Net.parameters(), lr=0.01)  
Time_Step = 28  

In [9]:
for epoch in range(100):
  for i, (image,label) in enumerate(TrainDL):
    Net.train()
    image= image.view(-1, Time_Step, InputNode).to(device)
    optim.zero_grad()
    output = Net(image)
    loss = LossFun(output, label.to(device))
    loss.backward()
    optim.step()
    if i % 50 == 49:
      Net.eval()       
      correct = 0
      for imgs, lbls in TestDL:
        imgs = imgs.view(-1, Time_Step, InputNode)
        outps = Net(imgs.to(device))
        correct += (torch.argmax(outps, dim=1) == lbls.to(device)).sum().item()
        accu = 100 * correct / len(lbls)
        display.clear_output(wait=True)
        print("回合：",epoch+1,"批次：",i+1,"損失函數",loss.item(),"準確度",accu)

回合： 100 批次： 600 損失函數 0.06483223289251328 準確度 97.94
